## Melhorando entendimento e acertos dos `datasets`

Neste notebook será apresentado algumas tarefas úteis que podem melhorar o entendimento do que é um "sinal bom" comparado a ruídos de dados em dados `SSVEP`.

Além disso, será listado uma série de melhorias que vão desde a etapa de extração de características até a classificação dos dados. Conhecemos estas melhorias como *tunning*.

### Visualização gráfica do sinal SSVEP

Para visualização do sinal SSVEP, vamos filtrar apenas o tempo em que houve estímulos para o `BETA dataset`.

### Melhoramentos das etapas de classificação

#### Extração de características

Basicamente, para levantamento das características, levamos em consideração apenas duas: `narrow-band SNR` e `wide-band SNR`. Vamos também adicionar duas outras características em cima dos dados estimulados no domínio da frequência (`FFT`): (1) amplitude dos targets e (2) média dos valores centrais dos targets.

Para cada característica estima-se que tenhamos a seguinte matriz: (160, 64, 40). Como teremos 4 características, a matriz final ficará com a seguinte dimensionalidade: (160, 64, 160) $\rightarrow$ (160, 10240).

**Observação**: Como os targets estão muito próximos um do outro, é possível que não tenha mais de um valor para calcular a média. Desta forma, será possível obter apenas a característica de "maior valor" ou amplitude do sinal no domínio da frequência.